In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter
import re
from wordcloud import WordCloud

In [2]:
# Cấu hình hiển thị
plt.style.use('default')  # Thay 'seaborn' bằng 'default'
sns.set_theme()  # Thêm dòng này
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Cấu hình font cho tiếng Việt
plt.rcParams['font.family'] = 'Arial Unicode MS'


In [3]:
plt.rcParams['font.family'] = 'Arial Unicode MS'

In [ ]:
print("Đang tải dataset...")
dataset = load_dataset("nguyendv02/ViMD_Dataset")

Đang tải dataset...


Repo card metadata block was not found. Setting CardData to empty.


Resolving data files:   0%|          | 0/103 [00:00<?, ?it/s]

train-00025-of-00103.parquet:  36%|###6      | 262M/725M [00:00<?, ?B/s]

d:\HK9\news\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\datasets--nguyendv02--ViMD_Dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00026-of-00103.parquet:   0%|          | 0.00/484M [00:00<?, ?B/s]

train-00027-of-00103.parquet:   0%|          | 0.00/417M [00:00<?, ?B/s]

train-00028-of-00103.parquet:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
print("\nKích thước dữ liệu:", df.shape)
print("\nThông tin các cột:")
df.info()


In [ ]:
print("\nMẫu dữ liệu:")
df.head()


In [ ]:
region_counts = df['region'].value_counts()
fig = px.pie(values=region_counts.values, 
             names=region_counts.index,
             title='Phân bố dữ liệu theo vùng miền',
             color_discrete_sequence=px.colors.qualitative.Set3)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()


In [ ]:
print("\nThống kê chi tiết theo vùng miền:")
print(region_counts)


In [ ]:
# ### 3.2. Phân bố theo giới tính

# %%
gender_counts = df['gender'].value_counts()
fig = px.bar(x=gender_counts.index, 
             y=gender_counts.values,
             title='Phân bố dữ liệu theo giới tính',
             labels={'x': 'Giới tính', 'y': 'Số lượng'},
             color=gender_counts.index,
             color_discrete_sequence=px.colors.qualitative.Set3)
fig.show()

In [ ]:
# In thống kê chi tiết
print("\nThống kê chi tiết theo giới tính:")
print(gender_counts)

In [ ]:
# ### 3.3. Phân bố độ dài audio theo vùng miền

# %%
fig = px.box(df, x='region', y='length',
             title='Phân bố độ dài audio theo vùng miền',
             labels={'region': 'Vùng miền', 'length': 'Độ dài (giây)'},
             color='region',
             color_discrete_sequence=px.colors.qualitative.Set3)
fig.show()


In [ ]:
print("\nThống kê độ dài audio theo vùng miền:")
print(df.groupby('region')['length'].describe())


In [ ]:
# 3.4. Phân bố số lượng từ theo vùng miền

# %%
df['word_count'] = df['text'].str.split().str.len()
fig = px.box(df, x='region', y='word_count',
             title='Phân bố số lượng từ theo vùng miền',
             labels={'region': 'Vùng miền', 'word_count': 'Số từ'},
             color='region',
             color_discrete_sequence=px.colors.qualitative.Set3)
fig.show()

In [ ]:
print("\nThống kê số lượng từ theo vùng miền:")
print(df.groupby('region')['word_count'].describe())

In [ ]:
# 4.1. Tạo Word Cloud cho từng vùng miền

# %%
def create_wordcloud(text, title):
    wordcloud = WordCloud(width=800, height=400,
                         background_color='white',
                         font_path='Arial Unicode MS',
                         max_words=100).generate(text)
    
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title)
    plt.show()

In [ ]:
# Tạo word cloud cho từng vùng miền
for region in df['region'].unique():
    region_text = ' '.join(df[df['region'] == region]['text'])
    create_wordcloud(region_text, f'Word Cloud - {region}')

In [ ]:
# 4.2. Phân tích từ vựng đặc trưng theo vùng miền

# %%
def get_word_freq_by_region(text_series, region):
    words = ' '.join(text_series).split()
    return pd.Series(words).value_counts()

region_word_freq = {}
for region in df['region'].unique():
    region_texts = df[df['region'] == region]['text']
    region_word_freq[region] = get_word_freq_by_region(region_texts, region)

# Hiển thị top 20 từ phổ biến nhất cho mỗi vùng miền
for region, freq in region_word_freq.items():
    print(f"\nTop 20 từ phổ biến nhất ở {region}:")
    print(freq.head(20))

In [ ]:
# 4.3. So sánh từ vựng giữa các vùng miền

# %%
def compare_regions(region1, region2, top_n=10):
    # Lấy top N từ phổ biến nhất của mỗi vùng
    words1 = set(region_word_freq[region1].head(top_n).index)
    words2 = set(region_word_freq[region2].head(top_n).index)
    
    # Tìm từ chung và từ riêng
    common_words = words1.intersection(words2)
    unique_to_1 = words1 - words2
    unique_to_2 = words2 - words1
    
    print(f"\nSo sánh {region1} và {region2}:")
    print(f"Từ chung: {', '.join(common_words)}")
    print(f"Từ đặc trưng của {region1}: {', '.join(unique_to_1)}")
    print(f"Từ đặc trưng của {region2}: {', '.join(unique_to_2)}")

# So sánh các vùng miền
compare_regions('North', 'Central')
compare_regions('Central', 'South')
compare_regions('North', 'South')

### Phân tích từ vựng đặc trưng của từng vùng

In [ ]:
def analyze_region_specific_words():
    # Loại bỏ các từ phổ biến (stopwords)
    stopwords = set(['là', 'của', 'và', 'trong', 'để', 'có', 'được', 'cho', 'với', 'tại'])
    
    for region in df['region'].unique():
        # Lấy tất cả từ của vùng
        region_words = ' '.join(df[df['region'] == region]['text']).split()
        # Loại bỏ stopwords
        region_words = [w for w in region_words if w not in stopwords]
        # Đếm tần suất
        word_freq = Counter(region_words)
        # Lấy top 10 từ đặc trưng
        print(f"\nTop 10 từ đặc trưng của vùng {region}:")
        print(word_freq.most_common(10))

# Phân tích độ dài từ theo vùng miền

In [ ]:
# Phân tích độ dài từ theo vùng miền
def analyze_word_length():
    df['avg_word_length'] = df['text'].apply(lambda x: np.mean([len(w) for w in x.split()]))
    
    fig = px.box(df, x='region', y='avg_word_length',
                 title='Phân bố độ dài trung bình của từ theo vùng miền',
                 labels={'region': 'Vùng miền', 'avg_word_length': 'Độ dài trung bình'})
    fig.show()

# Phân tích cấu trúc câu

In [ ]:
# Phân tích cấu trúc câu
def analyze_sentence_structure():
    # Đếm số từ trong mỗi câu
    df['sentence_length'] = df['text'].apply(lambda x: len(x.split()))
    
    # Vẽ biểu đồ phân bố độ dài câu
    fig = px.histogram(df, x='sentence_length', color='region',
                      title='Phân bố độ dài câu theo vùng miền',
                      labels={'sentence_length': 'Số từ trong câu', 'count': 'Số lượng'})
    fig.show()

# Phân tích từ vựng chung và riêng

In [ ]:
# Phân tích từ vựng chung và riêng
def analyze_common_and_unique_words():
    # Lấy top 100 từ phổ biến nhất của mỗi vùng
    region_top_words = {}
    for region in df['region'].unique():
        words = ' '.join(df[df['region'] == region]['text']).split()
        region_top_words[region] = set(Counter(words).most_common(100))
    
    # Tìm từ chung giữa các vùng
    common_words = set.intersection(*[set(dict(words).keys()) for words in region_top_words.values()])
    print("\nCác từ phổ biến chung giữa các vùng:")
    print(common_words)
    
    # Tìm từ riêng của mỗi vùng
    for region in df['region'].unique():
        other_regions = [r for r in df['region'].unique() if r != region]
        other_words = set.union(*[set(dict(region_top_words[r]).keys()) for r in other_regions])
        unique_words = set(dict(region_top_words[region]).keys()) - other_words
        print(f"\nCác từ đặc trưng của vùng {region}:")
        print(unique_words)

# Kết luận

In [ ]:

def draw_conclusions():
    print("""
    Kết luận từ phân tích:
    
    1. Phân bố từ vựng:
       - Vùng miền nào có nhiều từ đặc trưng nhất
       - Các từ phổ biến chung giữa các vùng
       - Sự khác biệt về độ dài từ và câu
    
    2. Đặc điểm ngôn ngữ:
       - Cấu trúc câu đặc trưng của mỗi vùng
       - Từ vựng chuyên ngành đặc trưng
       - Cách sử dụng từ ngữ địa phương
    
    3. Ứng dụng:
       - Phát triển hệ thống nhận dạng phương ngữ
       - Cải thiện công cụ dịch thuật
       - Phát triển ứng dụng học tiếng Việt theo vùng miền
    """)

In [ ]:
# Tạo biểu đồ thống kê tổng hợp giữa 3 vùng miền
def create_region_comparison_chart():
    # Tạo DataFrame tổng hợp
    comparison_data = pd.DataFrame({
        'Vùng miền': df['region'].unique(),
        'Số lượng mẫu': df['region'].value_counts().values,
        'Độ dài audio trung bình': df.groupby('region')['length'].mean().values,
        'Số từ trung bình': df.groupby('region')['word_count'].mean().values,
        'Độ dài từ trung bình': df.groupby('region')['text'].apply(
            lambda x: np.mean([len(w) for w in ' '.join(x).split()])).values
    })
    
    # Tạo biểu đồ radar (spider chart)
    fig = go.Figure()
    
    # Chuẩn hóa dữ liệu về thang điểm 0-1
    normalized_data = comparison_data.copy()
    for col in comparison_data.columns[1:]:
        normalized_data[col] = (comparison_data[col] - comparison_data[col].min()) / \
                             (comparison_data[col].max() - comparison_data[col].min())
    
    # Vẽ biểu đồ cho từng vùng miền
    for i, region in enumerate(comparison_data['Vùng miền']):
        fig.add_trace(go.Scatterpolar(
            r=normalized_data.iloc[i, 1:].values,
            theta=normalized_data.columns[1:],
            name=region,
            fill='toself'
        ))
    
    # Cập nhật layout
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1]
            )),
        showlegend=True,
        title='So sánh tổng hợp giữa ba vùng miền',
        template='plotly_white'
    )
    
    # Hiển thị biểu đồ
    fig.show()
    
    # In bảng thống kê chi tiết
    print("\nBảng thống kê chi tiết:")
    print(comparison_data.to_string(index=False))
    
    # In nhận xét
    print("\nNhận xét:")
    for col in comparison_data.columns[1:]:
        max_region = comparison_data.loc[comparison_data[col].idxmax(), 'Vùng miền']
        min_region = comparison_data.loc[comparison_data[col].idxmin(), 'Vùng miền']
        print(f"- {col}:")
        print(f"  + Vùng {max_region} có giá trị cao nhất: {comparison_data[col].max():.2f}")
        print(f"  + Vùng {min_region} có giá trị thấp nhất: {comparison_data[col].min():.2f}")